In [1]:
from qiskit import *
import math

# Qual é essa porta lógica quântica II - o Retorno

Você receberá uma operação que implementa uma transformação unitária em um único qubit: ela poderá ser ou a porta Y (com a possibilidade de uma fase global extra de -1) ou a sequência de portas Pauli Z e Pauli X (com a porta Z sendo aplicada primeiro e a X depois; com a possibilidade de uma fase global extra de -1).

Seu objetivo é identificar qual das operações foi usada apenas olhando para o resultado de sua aplicação.

Você deve usar o template da célula abaixo para apresentar sua solução implementando a função `qual_porta`. Nela, você deve retornar um número entre 0 e 3, apresentando qual é a porta lógica quântica passada no argumento `U`. Você deve retornar 

* 0 quando `U` for a porta Y,
* 1 quando `U` for a porta -XZ,
* 2 quando `U` for a porta -Y e
* 3 quando `U` for a porta XZ.

Você pode usar qualquer operação quântica para descobrir qual é a porta. O objetivo é usar seus conhecimentos de computação quântica para resolver este desafio, por isso, qualquer solução que não use computação quântica não será avaliada.


**Importante:** Essa questão deve ser resolvida usando o Qiskit. Qualquer solução que não use a ferramenta será desconsiderada.

In [2]:
def qual_porta(U : QuantumCircuit) -> int:
    
    answer = 0
    
    #############################
    # Escreva o seu código abaixo
    #############################
    
    # Ao contrário da primeira questão, nós não temos diferentes
    # operadores de Pauli. Nesse caso, os operadores fornecidos são os seguintes:
    # Y, iY, -Y, -iY. Portanto, vemos que temos o mesmo operadore, a menos
    # uma fase global. Para encontrar essa fase, utilizamos o algorítmo de Estimação 
    # de fase quântica, ou no inglês Quantum Phase Estimation (QPE). 
    # Devido ao problema possuir apenas fases que podem ser registradas com 
    # 2 digitos binários, escolhemos uma precisão de 2.
    
    precision = 2
    
    qc = QuantumCircuit(precision + 1, precision)
    
    # Coloca o estado da contagem em uma distribuição uniforme 
    # de probabilidade
    for qubit in range(precision):
        qc.h(qubit)

    # Preparamos os auto estados do operador Y.
    qc.h(precision)
    qc.s(precision)

    # Transformamos a unitária que queremos descobrir em uma porta
    # controlada.
    controlled_U = U.control()
    
    repetitions = 1
    for counting_qubit in range(precision):    
        for i in range(repetitions):
            qc.compose(controlled_U, [counting_qubit, precision], inplace=True)
        
        repetitions *= 2

    def qft_dagger(qc: QuantumCircuit, n: int):
        """ Apply the inverse of the Quantum Fourier Transform.
        """
        for qubit in range(n//2):
            qc.swap(qubit, n-qubit-1)
        for j in range(n):
            for m in range(j):
                qc.cp(-math.pi/float(2**(j-m)), m, j)
            qc.h(j)
    
    # Aplica a transformada inversa de Fourier Quântica
    qft_dagger(qc, 2)

    # Mede os qubits de contagem
    for n in range(precision):
        qc.measure(n,n)
    
    
    
    def circuit_simulation(shots: int) -> dict:
        """ run the circuit simulation for a given
        number of shots.

        Args:
            shots (int): number of shots

        Returns:
            dict: Dictionary of the shot count
        """
        aer_sim = Aer.get_backend('aer_simulator')
        t_qc = transpile(qc, aer_sim)
    
        results = aer_sim.run(t_qc, shots=shots).result()
        return results.get_counts()
    
    # Para esse caso, podemos utilizar apenas 1
    # shot, pois o estado vai colapsar para um das 
    # 4 bitstrings que representam a fase, no entanto
    # escolhemos colocar um número grande de shots para
    # demonstrar que isso é verdade
    shots = 8192
    answer = circuit_simulation(shots)
    
    # Converte a contagem para a base 10
    answer = list(answer.keys())[0]
    answer = int(answer, 2)
    
    return answer

> Não edite a partir daqui

## Teste sua solução 

Você pode usar o código abaixo para testar sua solução. Lembrando que, caso sua solução chegue ao resultado correto sem o uso de computação quântica, ela será desconsiderada, 

In [3]:
from random import shuffle

class Unitary:
    def __init__(self, num):
        self.qc = QuantumCircuit(1)
        
        if num == 0:
            self.Y()
        
        elif num == 1:
            self.mZX()
        
        elif num == 2:
            self.mY()
        
        elif num == 3:
            self.ZX()
    
    def Y(self):
        self.qc.y(0)

    def ZX(self):
        self.qc.z(0)
        self.qc.x(0)

    def mY(self):
        self.qc.z(0)
        self.qc.x(0)
        self.qc.y(0)
        self.qc.z(0)
        self.qc.x(0)

    def mZX(self):
        self.qc.x(0)
        self.qc.z(0)
 
     
def testar_solucao():
    tests = \
        [(Unitary(0), 0) for _ in range(10)] + \
        [(Unitary(1), 1) for _ in range(10)] + \
        [(Unitary(2), 2) for _ in range(10)] + \
        [(Unitary(3), 3) for _ in range(10)] 
    
    shuffle(tests)
    ok = 0
    for U, val in tests:
        solucao = qual_porta(U.qc)
        if val == solucao:
            ok += 1
    
    print(f"Solução {int((ok/len(tests))*100)}% correta")

In [4]:
testar_solucao()

Solução 100% correta
